# Classifying simulated events using a Convolutional Neural Network

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score, 
                             matthews_corrcoef, roc_curve, roc_auc_score)
from helper_functions import normalize_image_data, plot_history, plot_roc_auc
%load_ext autoreload
%autoreload 2

In [ ]:
# Load images and labels.
DATA_PATH = "../data/"

images = np.load(DATA_PATH+"images_training.npy")
labels = np.load(DATA_PATH+"labels_training.npy")

# Split the training indices into training and validation. 
# Validate with 25% of the data (default). Can be adjusted.
x_idx = np.arange(images.shape[0])
train_idx, val_idx, not_used1, not_used2 = train_test_split(x_idx, x_idx, test_size = 0.25)

## Reshaping data for CNNs
The convolutional layers we'll be using expect the inputs to have 4 dimensions:\
(samples, M, N, channels).\
M and N are the image dimensions, 16x16, but while RGB images have 3 channels, ours currently has 0, but should have 1.\
We solve this by just adding an empty axis.

In [ ]:
images = images.reshape(images.shape[0], 16, 16, 1)
print(images.shape)

# Model
Now, you can build your own network from scratch, and that's a useful exercise. We're going to skip that
here, and use one of the popular, exisiting frameworks that are widely used in current research.
The most used base frameworks are [TensorFlow](https://www.tensorflow.org/), [PyTorch](https://pytorch.org/), and [Keras](https://keras.io/). Keras is a high-level API that abstracts a large amount of the process of building,
training, and testing a model. You will need either TensorFlow or PyTorch, and the Keras API will automatically
detect which base framework you have.

## Build and compile
We base the initial model on a convolutional block in the [VGG16](https://arxiv.org/abs/1409.1556) architecture.

In [ ]:
# Instantiate the Sequential model, and add layers to it.
model = Sequential()

# Initial model based on VGG16 conv block, reduced to 32 filters
model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape= (16,16,1)))
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [ ]:
# Once the model is built, we need to compile it. This is where we specify the loss function,
# optimizer, and any metrics we need, even custom ones.

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
print(model.summary())

## Training
This is the point where we normalize our data, just as we pass it to the training function of the model.
The training run will display the progress as it goes through each batch.
$$ \text{num_batches} = \frac{\text{num_samples}}{\text{batch_size}}$$

In [ ]:
# Set parameters for the training.
batch_size = 32
epochs = 20

In [ ]:
# Setting validation data requires a tuple (val_input, val_targets). You can also just pass the
# entire training set without splitting, and specify validation_split instead of validation_data.
# The the model handles the splitting.

val_data = (normalize_image_data(images[val_idx]), labels[val_idx])
history = model.fit(
    x=normalize_image_data(images[train_idx]),
    y=labels[train_idx],
    validation_data=val_data,
    batch_size=batch_size,
    epochs=epochs,
)

# Evaluate the model

## Plot history of loss and accuracy

In [ ]:
# Use the function we stored in helper_functions.py
plot_history(history)

## Collection of metrics
* Accuracy
* Confusion Matrix
* F1-score
* Matthews Correlation Coefficient
* ROC-Curve and Area Under Curve

Check out the notebook on logistic regression for details around the metrics.

In [ ]:
# Predict on the validation set
pred = model.predict([normalize_image_data(images[val_idx])])
# Convert sigmoid values from prediction to integers for the metric functions
result = pred > 0.5

In [ ]:
accuracy = accuracy_score(labels[val_idx], result)
confmat = confusion_matrix(labels[val_idx], result)
f1 = f1_score(labels[val_idx], result)
mcc = matthews_corrcoef(labels[val_idx], result)

In [ ]:
# Print the metrics in an orderly fashion
print("Confusion matrix:\n", confmat)
print("Accuracy:", accuracy)
print("F1-score:", f1)
print("MCC:", mcc)

## ROC-curve and Area Under Curve
### All events

In [ ]:
plot_roc_auc(labels[val_idx], pred)